In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./recipe.csv', encoding='cp949', encoding_errors='ignore')
df.head()

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,10072,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,20070402131403
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,skfo0701,꽃날,5817,3,27,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내,20070402205937
2,128932,입안에서 톡톡톡,알밥,skfo0701,꽃날,6975,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내,20070402224355
3,131871,★현미호두죽,현미호두죽,cds1117,햇님&별님,3339,0,11,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,20070410142301
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,skfo0701,꽃날,7173,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,20070501000844


In [3]:
len(df['CKG_NM'].unique())

49680

In [4]:
df = df.drop_duplicates(subset='CKG_NM', keep='first')
len(df)

49680

In [6]:
import requests
import time
from bs4 import BeautifulSoup
import json

def food_info(name, retries=3, delay=2):
    '''
    This function gives you food information for the given input with retry logic.

    PARAMETERS
        - name(str): name of Korean food in Korean ex) food_info("김치찌개")
        - retries(int): number of retries in case of connection errors (default=3)
        - delay(int): seconds to wait before retrying (default=2)

    RETURN
        - res(dict): dict containing info for some Korean food related to 'name'
            - res['name'](str): name of food
            - res['ingredients'](str): ingredients to make the food
            - res['recipe'](list[str]): recipe instructions in order
    '''

    # Helper function to make requests with retry logic
    def make_request(url, retries, delay):
        for i in range(retries):
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    return response
                else:
                    print(f"HTTP response error: {response.status_code}")
            except requests.ConnectionError:
                print(f"ConnectionError occurred. Retrying {i+1}/{retries}...")
                time.sleep(delay)
        return None

    # Initial URL to search for the food
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = make_request(url, retries, delay)
    if not response:
        print(f"Failed to retrieve data after {retries} retries")
        return
    
    soup = BeautifulSoup(response.text, 'html.parser')
    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    
    if not food_list:
        print(f"No results found for {name}")
        return
    
    # Extract food ID and make a second request for the recipe details
    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = make_request(new_url, retries, delay)
    if not new_response:
        print(f"Failed to retrieve data after {retries} retries")
        return
    
    soup = BeautifulSoup(new_response.text, 'html.parser')
    food_info = soup.find(attrs={'type':'application/ld+json'})
    
    if food_info is None:
        print(f"No structured data found for {name}")
        return
    
    try:
        result = json.loads(food_info.text)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        return

    # Check for ingredients and recipe instructions
    ingredient_list = result.get('recipeIngredient', None)
    if ingredient_list:
        ingredients = ', '.join(ingredient_list)
    else:
        ingredients = 'No ingredients available'
    
    recipe_list = result.get('recipeInstructions', None)
    if recipe_list:
        recipe = [step.get('text', 'No instructions available') for step in recipe_list]
        recipe = [f'{i+1}. {step}' for i, step in enumerate(recipe)]
    else:
        recipe = ['No recipe available']
    
    res = {
        'name': name,
        'ingredients': ingredients,
        'recipe': recipe
    }
    
    return res

In [7]:
results = []
# CKG_NM 열의 값들을 하나씩 반복하여 food_info 호출
for food_name in df['CKG_NM'].iloc[15000:20000]:
    food_data = food_info(food_name)
    if food_data:  # food_info가 None이 아닌 경우에만 추가
        results.append(food_data)

result_df = pd.DataFrame(results)

JSON parsing error: Invalid control character at: line 5 column 26 (char 89)
JSON parsing error: Invalid control character at: line 5 column 29 (char 92)
ConnectionError occurred. Retrying 1/3...
JSON parsing error: Invalid control character at: line 18 column 1046 (char 1765)
JSON parsing error: Invalid control character at: line 5 column 29 (char 92)
ConnectionError occurred. Retrying 1/3...
No structured data found for 강낭콩찐빵
ConnectionError occurred. Retrying 1/3...
JSON parsing error: Invalid control character at: line 5 column 31 (char 94)
JSON parsing error: Expecting ',' delimiter: line 23 column 2114 (char 2860)
JSON parsing error: Invalid control character at: line 5 column 38 (char 101)
JSON parsing error: Invalid control character at: line 23 column 1985 (char 2792)
JSON parsing error: Expecting ',' delimiter: line 23 column 2342 (char 3209)
JSON parsing error: Invalid control character at: line 5 column 33 (char 96)
ConnectionError occurred. Retrying 1/3...
JSON parsing err

In [8]:
result_df.head()

,name,ingredients,recipe
0,파소스,"파 하얀부분 5대 , 참기름 종이컵기준 1/2컵, 가는소금 0.5t, 일반소금 1T...","[1. 파는 깨끗하게 씻어서 칼로 십자모양을 내서 송송 잘라줍니다, 2. 송송 잘라..."
1,굴소스카레볶음밥,"양파 1/2개, 당근 조금, 호박 조금, 청양고추 1개, 햄 조금, 계란 1개, 굴...","[1. 재료를 준비해주세요^^, 2. 후라이판에 기름 넣으시구 양파와 파를 넣고 볶..."
2,쫄짜장,"떡볶이 떡 2컵, 어묵 4장, 햄 1컵 (or 소세지), 양파 1/2개, 물 3컵 ...","[1. 물 3컵을 넣고 끓여준 후 다진마늘을 반스푼정도 넣어주세요., 2. 물이 끓..."
3,잔대나물무침,"잔대나물 1단, 소금 1T, 고추장 1T, 다진마늘 1/2T, 간장 1/2T, 참기...","[1. 먼저 잔대의 억센 줄기 부분과 상태가좋지 않은 잎은 제거 합니다., 2. 흐..."
4,깻잎멸치볶음,"깻잎 80장, 양파 작은거 3개, 양파 큰거 2개, 멸치 2종이컵, 다진마늘` 1숟...",[1. 여름철에 시골에서 들깻잎을 데쳐서 냉동 보관을 해놓아요.데쳐 놓은 깻잎이랑(...


### recipe   
[] , \n, 'No recipe available' 삭제

In [10]:
result_df['recipe'] = result_df['recipe'].apply(lambda x: '\n'.join(x).replace('[','').replace(']',''))

In [11]:
def clean_recipe(recipe_str):
    # 숫자와 점(예: '1.', '2.')을 제거하고, 줄바꿈 문자를 모두 제거하는 정규 표현식
    cleaned = recipe_str.replace('\n', ' ') # 줄바꿈 문자를 공백으로 대체
    return cleaned

In [12]:
result_df['recipe'] = result_df['recipe'].apply(clean_recipe)

In [13]:
result_df = result_df[~result_df['recipe'].apply(lambda x: 'No recipe available' in x)].reset_index(drop=True)

### ingredients


In [15]:
import re
def clean_ingredients(ingredient_str):
    # 숫자+한글, 숫자+영어, "1/2개" 등 제거하는 정규 표현식
    cleaned = re.sub(r'\d+/\d+[가-힣]*|\d+[a-zA-Z가-힣]*|약간|적당히', '', ingredient_str)
    # 쉼표로 나누고 공백 제거
    cleaned_list = [item.strip() for item in cleaned.split(',') if item.strip()]
    return ', '.join(cleaned_list)

In [16]:
result_df['ingredients'] = result_df['ingredients'].apply(clean_ingredients)

In [17]:
result_df.head()

,name,ingredients,recipe
0,파소스,"파 하얀부분, 참기름 종이컵기준, 가는소금 ., 일반소금, 통깨",1. 파는 깨끗하게 씻어서 칼로 십자모양을 내서 송송 잘라줍니다 2. 송송 잘라준 ...
1,굴소스카레볶음밥,"양파, 당근 조금, 호박 조금, 청양고추, 햄 조금, 계란, 굴소스, 카레",1. 재료를 준비해주세요^^ 2. 후라이판에 기름 넣으시구 양파와 파를 넣고 볶아주...
2,쫄짜장,"떡볶이 떡, 어묵, 햄 (or 소세지), 양파, 물 (), 설탕, 다진마늘 .,...",1. 물 3컵을 넣고 끓여준 후 다진마늘을 반스푼정도 넣어주세요. 2. 물이 끓는 ...
3,잔대나물무침,"잔대나물, 소금, 고추장, 다진마늘 T, 간장 T, 참기름, 볶음참깨, 소금",1. 먼저 잔대의 억센 줄기 부분과 상태가좋지 않은 잎은 제거 합니다. 2. 흐르는...
4,깻잎멸치볶음,"깻잎, 양파 작은거, 양파 큰거, 멸치, 다진마늘`, 통깨, 들기름, 간장",1. 여름철에 시골에서 들깻잎을 데쳐서 냉동 보관을 해놓아요.데쳐 놓은 깻잎이랑(생...


In [18]:
result_df['recipe'][0]

'1. 파는 깨끗하게 씻어서 칼로 십자모양을 내서 송송 잘라줍니다 2. 송송 잘라준 파를 볼에 담아주시면 되요 3. 참기름 종이컵 반컵 . 가는소금(아이스크림숟가락)0.5 / 일반소금 1수저  * 가는소금과 일반소금은 1 : 1 기준이라는데 저는 가는소금이 더 간이 잘베는거라 조금만 넣었어요  통깨는 밥수저로 3숟가락 4. 파가 기름에 잠길정도로 맞추라지만  반컵만 넣어도 충분해요 5. 담아두면 기름이 밑으로 잘 내려가니까  걱정하지 마세요 ~'

In [37]:
result_df.to_excel('./recipe_result.xlsx', index=False)